## 爬虫代码
- **从2020-1-13至Yesterday疫情人数**

In [1]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import re 
from bs4 import BeautifulSoup  
from datetime import datetime

#  国内疫情数据
url = 'https://view.inews.qq.com/g2/getOnsInfo?name=disease_other'
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.25 Safari/537.36 Core/1.70.3765.400 QQBrowser/10.6.4148.400"}
response = requests.get(url,headers=headers)

a = response.text  # 为字符串json数据格式，需要将其转化成json格式
data = json.loads(a) # 转化成json格式
data = json.loads(data["data"])
data = data["chinaDayList"] # 每天的data数据取出来

df = pd.DataFrame(data)
df.drop(columns=['nowConfirm', 'nowSevere','importedCase', 'deadRate', 'healRate', 'noInfect'],inplace=True)
df["date"] = "2020-"+df["date"].map(lambda x:x.replace(".","-"))
df.to_csv(r'./file/math_model.csv',mode='a',encoding='gbk')

## 数据指标SQL

In [2]:
SELECT 
    DATE_FORMAT(付款时间,'%Y-%m-%d') AS 时间,
    COUNT(订单ID) AS 订单量,
    count(distinct 用户ID) as 用户量,
    sum(if(是否是首单='是',1,0)) as 新增用户量,
    round(sum(实收金额),2) AS 实收,
    sum(优惠金额) as 优惠,
    sum(优惠金额+实收金额) as 应收,
    concat(round(sum(优惠金额)/sum(优惠金额+实收金额),2)*100,'%') as 优惠金额占比,
    round(sum(实收金额)/COUNT(订单ID)) as 单均收益
FROM kkb.order
where DATE_FORMAT(付款时间,'%Y-%m-%d') >= '2020-02-01' and DATE_FORMAT(付款时间,'%Y-%m-%d') <= '2020-02-10'
GROUP BY DATE_FORMAT(付款时间,'%Y-%m-%d');

IndentationError: unexpected indent (<ipython-input-2-d87d0d499682>, line 2)

## 连续活跃用户SQL

In [ ]:
select 用户ID
    ,max(days) incessancy_days
    ,min(pay_dd) start_date
    ,max(pay_dd) end_date
from
(select 用户ID
        ,@cont_day :=
	   (case 
        when (@last_uid = 用户ID and DATEDIFF(pay_date,@last_dt)=1) then (@cont_day + 1)
		when (@last_uid = 用户ID and DATEDIFF(pay_date,@last_dt)<1) then (@cont_day + 0)
		else 1
		end
        ) as days
        ,(@cont_ix := (@cont_ix + if(@cont_day = 1, 1, 0))) AS cont_ix
        ,@last_uid := 用户ID
        ,@last_dt := pay_date as pay_dd
    from
    (select 用户ID,DATE_FORMAT(付款时间,'%Y-%m-%d') as pay_date
        from kkb.order
        where DATE_FORMAT(付款时间,'%Y-%m-%d') >= '2020-01-01' and DATE_FORMAT(付款时间,'%Y-%m-%d') <= '2020-08-06'
        group by 用户ID,DATE_FORMAT(付款时间,'%Y-%m-%d')
        order by 用户ID
    ) as t1,
    (select @last_uid := '',
			@last_dt  := '',
			@cont_ix  := 0,
			@cont_day := 0
    ) as t2
) as t
group by 用户ID
order by incessancy_days desc;